<a href="https://colab.research.google.com/github/mmegias/IA3-UV/blob/main/DL_I_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perceptrón Multi-Capa

En este *Notebook* vamos a implementar un Perceptron Multi-Capa (MLP: *Multilayer Perceptron*) para una tarea de clasificación: 

Para ello, vamos a crear nuestro propio conjunto de datos sintético

**RECUERDA**

En el perceptron multicapa, la salida de cada capa es calculada usando las activaciones de las capas anteriores de la siguiente manera.

$$h_{i} = \sigma(W_i h_{i-1} + b_i)$$

Dónde ${h}_i$ es el vector de activación de la $i$-ésima capa (o los datos de entrada para $i=0$), ${W}_i$ y ${b}_i$ son la matriz de pesos y el vector de sesgo (*bias*) para la capa $i$-ésima respectivamente

<br><rb>
$\sigma(\cdot)$ es la función de activación is the activation function. En nuestro ejemplo usaremos la función de activación *Sigmoide* para las capas ocultas (intermedias) y la *softmax* para la última capa, dado que se trata de un problema de clasificación.


## Creación del conjunto de datos sintético
Lo primero que vamos a hacer es crear una función para generar datos sintéticos aletorios. Debes rellenar el siguiente código. Para ello, crea 2 nubes de puntos de 3 dimensiones (una para casa clase). La primera nube (800 puntos) que provenga de una distribución normal con media 0 y desviación estándar 1. La segunda nube de puntos (500 puntos), que sea igual que la primera pero que la última componente tenga media 3 y desviación estándar 0.5. Debes crear un vector de salida con valor 0 para la primera nube de puntos y 1 para la segunda. 

In [91]:
import numpy as np
import sys
import matplotlib.pyplot as plt

def generate_dataset(output_dim=3, num_examples=[800, 500]):
    # Tu código aquí
    #x1= 1*np.random.randn(num_examples[0],output_dim)+ 0 
    #x2= 1*np.random.randn(num_examples[1],output_dim)+ 0 
    #x2[:,2] = 0.5*np.random.randn(num_examples[1],1) + 3
    #x1_label= np.zeros([num_examples[0],1])
    #x2_label= np.ones([num_examples[1],1])
    #x= np.concatenate((x1,x2), axis=0)
    #y= np.concatenate((x1_label,x2_label), axis=0)
    features_class_1 = np.random.randn(num_examples[0], output_dim)
    features_class_2 = np.random.randn(num_examples[1], output_dim)
    features_class_2[:, 2] = np.random.randn(num_examples[1]) * 0.5 + 3
    labels_class_1 = np.ones([num_examples[0], 1])
    labels_class_2 = np.zeros([num_examples[1], 1])
    x = np.concatenate((features_class_1, features_class_2), axis=0)
    y = np.concatenate((labels_class_1, labels_class_2), axis=0)
    return (x, y)

Ahora vamos a llamar a la función anterior para generar nuestro conjunto de datos:

In [93]:
np.random.seed(1)
num_examples = [800,500]
output_dim = 3
x, y = generate_dataset( output_dim=output_dim, num_examples=num_examples)

A continuación, representa los datos en 3D, con diferentes colores para representar la pertenencia a cada una de las clases:

## Función de activación y su derivada

A continuación vamos a crear dos métodos. El primero implementará la función de activación correspondiente a la sigmoide. Esta función es necesaria para el cáculo *forward pass* de la red.  Dado un valor "x", esta función deberá cacular el varlor con respecto a la siguiente fórmula:
$$a(x) =  \frac{\mathrm{1} }{\mathrm{1} + e^{-x}}$$

El segundo método, implementará la derivada de la función anterior, necesaria para el *backward pass*. Procediendo con la derivada, obtenemos las siguiente fórmula:

$${a}' = a(1-a)$$



In [94]:
def sigmoid(x):
         return 1. / (1 + np.exp(-x))

In [95]:
def sigmoid_derivada(x):
    return sigmoid(x)*(1.0-sigmoid(x))

## Contruyendo la Red Neuronal Artificial
A continuación vamos a definir la clase Layer. Ésta implementará los 3 métodos necesarios para entrenar la red neuronal (*forward*, *backward* y *update*). Recuerda que el `__init__` debe generar los pesos ${W}$ y ${b}$  de manera aleatoria. Para ello, utiliza una distribución normal con media -0.1 y desviación estándar 0.2.

<br><rb>
Para el cáculo del *backward pass* hay que calcular las derivadas parciales del coste (error) con respecto a cada uno de los parámetros de la red (${W}$ y ${b}$). Para ello, en esta clase solamente vamos a declarar las derivadas de las capas intermedias (las derivadas parciales de la última capa, que se calculan de manera distinta, las calcularemos fuera de esta clase. Los $\delta$ de los ${W}$ se calculan en función de los $\delta$ de la capa anterior:

$$ \delta^{l-1} = W^{l}\delta^{l}\frac{\partial a^{l-1}}{\partial z^{l-1}}$$

Donde $a$ es la función de activación, $z$ es la salida de la suma ponderada de la entrada (la salida de la neurona, la regresión lineal, antes de aplicar la función de activación) y $l-1$ hace referencia a la "*Layer*" anterior.

<br><rb>
 De ahí, que el método `backward` tenga compo parámetro de entrad la variable `output_delta` (correspondiente a los $\delta$ de la capa anterior). Para el cálculo de $\delta$ de los parámetros ${W}$ la capa actual hemos de realizar dos operaciones. La primera es multiplicar el $\delta$ de la capa posterior (que proviene del parámetro de entrada `output_delta`) por la derivada de la función de activación aplicada sobre la salida de la red en el *forward pass*. La segunda operación es el producto del resultado anterior por los pesos ${W}$.

<br><rb>
Para actualizar la red (método `update`) deberemos actualizar los pesos ${W}$ con un nuevo valor de la siguiente manera:
$$W_{i}(t)\:=\:W_{i}(t-1) - \alpha(\frac{\partial error}{\partial W_{i}})$$

Siendo $(\frac{\partial error}{\partial W_{i}})$ la derivada de la función de activación aplicada sobre la salida de la red en el forward pass (calculada en el método `backward`).


In [96]:

class Layer(object):
    def __init__(self, input_dim, output_dim, activation_func, act_func_deriv):
        self.weights= np.random.randn(input_dim,output_dim)*0.2-0.1
        self.bias=np.random.randn(output_dim)*0.2-0.1
        self.activation_func= activation_func
        self.act_func_deriv= act_func_deriv

    def forward(self, input):
        self.input = input
        self.output = self.activation_func(np.dot(input,self.weights) + self.bias)
        return self.output

    def backward(self, output_delta):
        self.weight_output_delta = output_delta * self.act_func_deriv(self.output) #funcion de coste= diferencia entre lo "y" esperada y "y" calculada
        new_delta = np.dot(self.weight_output_delta, self.weights.T)
        return new_delta

    def update(self, learning_rate=0.1):
      self.weights -= np.dot(self.input.T, self.weight_output_delta) * learning_rate


## Entrenamiento de la Red

Inicializamos  los siguientes parámetros:


In [97]:

batch_size = 10
learning_rate = 0.1

input_dim = len(x[0])
layer_1_dim = 128
layer_2_dim = 64
output_dim = 1
iterations = 1000



Creamos nuestra red neuronal de 3 capas llamando a la clase `Layer` tantas veces como sea necesario:


In [99]:
layer1= Layer(input_dim, layer_1_dim, sigmoid, sigmoid_derivada)
layer2= Layer(layer_1_dim,layer_2_dim,sigmoid_derivada,sigmoid_derivada)
layer3= Layer(layer_2_dim, output_dim, sigmoid_derivada,sigmoid_derivada)

A continuación procederemos a entrenar nuestra red. Para ello necesitaremos crear *batches* de nuestro conjunto de datos para ir pasándoselos a la red por lotes. Para ello necesitaremos 2 bucles `for`. El primero iterará sobre un número de iteraciones (épocas), y el segundo sobre los *batches*. Se pasará cada *batch* a la red para hacer el *forward pass* y calcular la salida de la red. Después, haremos el *backward pass* (en order inverso) para actualizar los pesos de la red. Recuerda, que en la implementación de la clase `Layer`solamente se implemetó el *backward pass* de las capas intermedias. Por lo tanto, es necesario que prinero calcules el $\delta$ de la última capa para pasárselo al método `Layer.backward()`. Dicho $\delta$ es igual a la diferencia entre la salida de la red y la salida real. Finalmente actualiza cada capa con el método `Layer.update()` y computa el error acumulado por cada batch en cada época. Imprime el error cada 10 épocas por pantalla.

In [106]:
for iter in range(iterations):
    error = 0

    for batch_i in range(int(len(x) / batch_size)):
        batch_x = x[(batch_i * batch_size):(batch_i + 1) * batch_size]
        batch_y = y[(batch_i * batch_size):(batch_i + 1) * batch_size]

        layer1_out = layer1.forward(batch_x)
        layer2_out = layer2.forward(layer1_out)
        layer3_out = layer3.forward(layer2_out)

        layer3_delta = layer3_out - batch_y
        layer2_delta = layer3.backward(layer3_delta)

        error= 